In [67]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [68]:
import os
import keras
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras import models
from keras.callbacks import EarlyStopping, CSVLogger, LearningRateScheduler, ModelCheckpoint
from keras.layers import Dense, Flatten, Input, Dropout, BatchNormalization
from keras.models import load_model, Model
from keras.optimizers import Adam
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

In [69]:
path='/content/drive/MyDrive/Capstone_Design/EEG_DATA_Shuffle_Train_500.csv'
dataDF = pd.read_csv(path)
dataDF = dataDF.drop('Unnamed: 0', axis=1)
dataDF = dataDF[dataDF.iloc[:, -1] != 3]
dataDF

,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,-613.9531,8.3703,-176.9841,-219.4107,-0.5328,94.9314,-39.4071,130.4078,-113.2378,-4.4257,...,-84.1440,126.1760,-23.1316,-34.8843,-114.5272,-617.7455,-127.1501,-235.2947,-181.4069,0.0
2,19.2428,39.7509,45.2220,22.9375,43.3171,52.6086,37.2702,7.9396,45.4657,10.6151,...,11.0563,42.4707,14.7642,4.5331,26.1308,36.0465,47.0181,6.1247,-21.7909,0.0
4,-9.8380,-14.6411,-17.4022,-23.3906,-5.4970,-5.4633,-5.6305,-27.2574,-8.3492,-10.9784,...,-18.1588,-14.0856,-11.9353,-13.7297,-9.6201,-21.1847,-14.3220,-10.4927,16.7754,0.0
5,-10.6741,16.9807,0.9579,-7.3524,-2.5546,9.8229,6.0904,6.3204,-17.4586,7.6739,...,47.4403,29.7741,24.5950,30.7687,32.5438,0.3632,8.0525,26.8370,53.7960,1.0
6,42.6648,42.0825,77.7059,45.6584,36.0783,28.5225,37.0225,41.0220,46.5493,-41.1988,...,-12.9571,-18.7840,-22.7828,-15.6432,-52.4583,-3.6135,-2.8635,-14.3334,5.2327,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2152,37.3678,28.2781,31.7802,8.6374,0.2593,21.5804,20.2608,17.8901,172.0313,25.6119,...,21.3070,48.4440,11.8208,-10.2947,41.1409,-78.3884,2.6567,34.2597,19.4901,1.0
2154,49.7153,43.8895,38.9049,30.7865,54.5577,17.6120,40.5788,-1.8589,20.7679,-67.7020,...,-14.1029,-30.6808,-151.8023,18.6433,-46.9215,-52.4163,-7.1656,-21.5339,5.3341,1.0
2155,14.3461,-1.6934,-21.1162,-12.7199,-10.6491,-5.0185,-13.5567,-7.7491,-5.5391,24.0267,...,28.5637,-24.4292,-26.2400,-12.7488,-1.2411,43.8751,14.2213,-28.3929,-16.0004,1.0
2156,-5.8924,21.1051,3.8857,0.6180,5.5196,31.6711,17.1194,12.5737,37.4985,4.4398,...,2.8834,-14.6937,-13.9947,-1.7489,-11.3992,8.0760,-10.1007,-11.8973,-22.9405,1.0


In [70]:
mapping_dict = {idx: i for i, idx in enumerate(dataDF.index[:1621])}
dataDF.index = dataDF.index.map(mapping_dict)
dataDF

,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,-613.9531,8.3703,-176.9841,-219.4107,-0.5328,94.9314,-39.4071,130.4078,-113.2378,-4.4257,...,-84.1440,126.1760,-23.1316,-34.8843,-114.5272,-617.7455,-127.1501,-235.2947,-181.4069,0.0
1,19.2428,39.7509,45.2220,22.9375,43.3171,52.6086,37.2702,7.9396,45.4657,10.6151,...,11.0563,42.4707,14.7642,4.5331,26.1308,36.0465,47.0181,6.1247,-21.7909,0.0
2,-9.8380,-14.6411,-17.4022,-23.3906,-5.4970,-5.4633,-5.6305,-27.2574,-8.3492,-10.9784,...,-18.1588,-14.0856,-11.9353,-13.7297,-9.6201,-21.1847,-14.3220,-10.4927,16.7754,0.0
3,-10.6741,16.9807,0.9579,-7.3524,-2.5546,9.8229,6.0904,6.3204,-17.4586,7.6739,...,47.4403,29.7741,24.5950,30.7687,32.5438,0.3632,8.0525,26.8370,53.7960,1.0
4,42.6648,42.0825,77.7059,45.6584,36.0783,28.5225,37.0225,41.0220,46.5493,-41.1988,...,-12.9571,-18.7840,-22.7828,-15.6432,-52.4583,-3.6135,-2.8635,-14.3334,5.2327,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1615,37.3678,28.2781,31.7802,8.6374,0.2593,21.5804,20.2608,17.8901,172.0313,25.6119,...,21.3070,48.4440,11.8208,-10.2947,41.1409,-78.3884,2.6567,34.2597,19.4901,1.0
1616,49.7153,43.8895,38.9049,30.7865,54.5577,17.6120,40.5788,-1.8589,20.7679,-67.7020,...,-14.1029,-30.6808,-151.8023,18.6433,-46.9215,-52.4163,-7.1656,-21.5339,5.3341,1.0
1617,14.3461,-1.6934,-21.1162,-12.7199,-10.6491,-5.0185,-13.5567,-7.7491,-5.5391,24.0267,...,28.5637,-24.4292,-26.2400,-12.7488,-1.2411,43.8751,14.2213,-28.3929,-16.0004,1.0
1618,-5.8924,21.1051,3.8857,0.6180,5.5196,31.6711,17.1194,12.5737,37.4985,4.4398,...,2.8834,-14.6937,-13.9947,-1.7489,-11.3992,8.0760,-10.1007,-11.8973,-22.9405,1.0


In [71]:
dataDF

,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,-613.9531,8.3703,-176.9841,-219.4107,-0.5328,94.9314,-39.4071,130.4078,-113.2378,-4.4257,...,-84.1440,126.1760,-23.1316,-34.8843,-114.5272,-617.7455,-127.1501,-235.2947,-181.4069,0.0
1,19.2428,39.7509,45.2220,22.9375,43.3171,52.6086,37.2702,7.9396,45.4657,10.6151,...,11.0563,42.4707,14.7642,4.5331,26.1308,36.0465,47.0181,6.1247,-21.7909,0.0
2,-9.8380,-14.6411,-17.4022,-23.3906,-5.4970,-5.4633,-5.6305,-27.2574,-8.3492,-10.9784,...,-18.1588,-14.0856,-11.9353,-13.7297,-9.6201,-21.1847,-14.3220,-10.4927,16.7754,0.0
3,-10.6741,16.9807,0.9579,-7.3524,-2.5546,9.8229,6.0904,6.3204,-17.4586,7.6739,...,47.4403,29.7741,24.5950,30.7687,32.5438,0.3632,8.0525,26.8370,53.7960,1.0
4,42.6648,42.0825,77.7059,45.6584,36.0783,28.5225,37.0225,41.0220,46.5493,-41.1988,...,-12.9571,-18.7840,-22.7828,-15.6432,-52.4583,-3.6135,-2.8635,-14.3334,5.2327,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1615,37.3678,28.2781,31.7802,8.6374,0.2593,21.5804,20.2608,17.8901,172.0313,25.6119,...,21.3070,48.4440,11.8208,-10.2947,41.1409,-78.3884,2.6567,34.2597,19.4901,1.0
1616,49.7153,43.8895,38.9049,30.7865,54.5577,17.6120,40.5788,-1.8589,20.7679,-67.7020,...,-14.1029,-30.6808,-151.8023,18.6433,-46.9215,-52.4163,-7.1656,-21.5339,5.3341,1.0
1617,14.3461,-1.6934,-21.1162,-12.7199,-10.6491,-5.0185,-13.5567,-7.7491,-5.5391,24.0267,...,28.5637,-24.4292,-26.2400,-12.7488,-1.2411,43.8751,14.2213,-28.3929,-16.0004,1.0
1618,-5.8924,21.1051,3.8857,0.6180,5.5196,31.6711,17.1194,12.5737,37.4985,4.4398,...,2.8834,-14.6937,-13.9947,-1.7489,-11.3992,8.0760,-10.1007,-11.8973,-22.9405,1.0


In [72]:
ct = 500

x_train = []
y_train = []

print(len(dataDF))

for i in range(len(dataDF)):
    print(i)
    x_train.append([])
    x = 0
    y = 20
    one_row_data = dataDF.iloc[i]
    y_train.append(one_row_data[y - 1])
    for j in range(ct):
        x_train[i].append([one_row_data[x:y - 1]])
        x += 20
        y += 20

x_train = np.array(x_train).reshape(len(x_train), ct, 19)
y_train = np.array(pd.get_dummies(y_train))

1620
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
2

In [73]:
print("Training set shapes:")
print(x_train.shape)
print(y_train.shape)

Training set shapes:
(1620, 500, 19)
(1620, 3)


In [74]:
# from scipy.signal import resample

# def augment_data(x, y):
#     augmented_x = []
#     augmented_y = []

#     for i in range(len(x)):
#         augmented_x.append(x[i])
#         augmented_y.append(y[i])

#         # 데이터를 뒤집기
#         flipped_x = np.flip(x[i], axis=0)
#         augmented_x.append(flipped_x)
#         augmented_y.append(y[i])

#         # 노이즈 추가
#         noisy_x = x[i] + np.random.normal(0, 0.1, x[i].shape)
#         augmented_x.append(noisy_x)
#         augmented_y.append(y[i])

#         # 주파수 변환 (예시: 500Hz에서 250Hz로 다운샘플링)
#         downsampled_x = resample(x[i], int(x[i].shape[0] / 2))

#         # 주파수 변환 후 크기가 변경되므로, 기존 데이터의 크기와 동일하게 맞춰줍니다.
#         downsampled_x = np.pad(downsampled_x, ((0, x[i].shape[0] - downsampled_x.shape[0]), (0, 0)), mode='constant')

#         augmented_x.append(downsampled_x)
#         augmented_y.append(y[i])

#     return np.array(augmented_x), np.array(augmented_y)

# augmented_x_train, augmented_y_train = augment_data(x_train, y_train)

# print("\n데이터 증강 후 shape:")
# print("x_train shape:", augmented_x_train.shape)
# print("y_train shape:", augmented_y_train.shape)

In [75]:
# def augment_data(x, y):
#     augmented_x = []
#     augmented_y = []

#     for i in range(len(x)):
#         augmented_x.append(x[i])
#         augmented_y.append(y[i])

#         # 데이터를 뒤집기
#         flipped_x = np.flip(x[i], axis=0)
#         augmented_x.append(flipped_x)
#         augmented_y.append(y[i])

#         # 노이즈 추가
#         noisy_x = x[i] + np.random.normal(0, 0.1, x[i].shape)
#         augmented_x.append(noisy_x)
#         augmented_y.append(y[i])

#     return np.array(augmented_x), np.array(augmented_y)

# augmented_x_train, augmented_y_train = augment_data(x_train, y_train)

# print("\n데이터 증강 후 shape:")
# print("x_train shape:", augmented_x_train.shape)
# print("y_train shape:", augmented_y_train.shape)

In [76]:
# # Define data augmentation functions
# def add_noise(data, noise_level=0.05):
#     noise = np.random.normal(scale=noise_level, size=data.shape)
#     return data + noise

# def apply_random_scaling(data, min_scale=0.9, max_scale=1.1):
#     scale = np.random.uniform(min_scale, max_scale, size=(data.shape[0], 1, 1))
#     return data * scale

# def apply_random_translation(data, max_translation=10):
#     translation = np.random.randint(-max_translation, max_translation, size=data.shape)
#     return np.roll(data, translation, axis=1)

# # Define data augmentation function
# def augment_data(x_train, y_train, num_augmentations=3):
#     augmented_x = []
#     augmented_y = []

#     for i in range(len(x_train)):
#         original_x = x_train[i]
#         original_y = y_train[i]

#         # Add original data
#         augmented_x.append(original_x)
#         augmented_y.append(original_y)

#         # Apply data augmentation
#         for _ in range(num_augmentations):
#             # Apply random transformations
#             augmented_x.append(apply_random_translation(apply_random_scaling(add_noise(original_x))))
#             augmented_y.append(original_y)

#     return np.array(augmented_x), np.array(augmented_y)

# # Perform data augmentation
# augmented_x_train, augmented_y_train = augment_data(x_train, y_train)

# # Check the shape of augmented data
# print("Augmented x_train shape:", augmented_x_train.shape)
# print("Augmented y_train shape:", augmented_y_train.shape)

In [77]:
path='/content/drive/MyDrive/Capstone_Design/EEG_DATA_Shuffle_Validation_500.csv'
dataDF = pd.read_csv(path)
dataDF = dataDF.drop('Unnamed: 0', axis=1)
dataDF = dataDF[dataDF.iloc[:, -1] != 3]
dataDF

,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,17.8702,21.4625,7.5565,-27.3715,2.4686,4.5210,-4.7260,-12.1676,2.2848,-16.4927,...,16.4425,27.0572,-11.4758,4.2899,27.0235,-7.7163,-24.5729,3.5688,-1.5061,1.0
1,-34.6827,-13.2270,28.4124,37.2452,-28.3499,-30.9165,18.4352,-21.4872,18.3771,27.5069,...,-30.0969,13.4330,27.0925,-14.5877,46.8918,3.2198,19.4962,38.6907,-53.4658,0.0
3,-76.7901,10.4661,60.9218,39.1930,33.3773,65.0877,49.0782,5.1342,43.5867,0.9796,...,-38.8314,-50.8898,-42.2924,-22.9204,-48.7766,-44.7011,-38.4557,-40.1042,-36.8672,1.0
4,4.3084,92.5663,84.6904,46.8202,48.0335,51.5039,53.3825,32.5205,93.0373,2.1225,...,14.2065,10.0970,24.9994,33.3981,-4.9856,43.4911,34.7056,-2.6518,-3.7837,1.0
5,1.3368,-22.5723,15.9681,-59.4414,-10.9613,-30.8094,-60.6815,-26.9298,220.6445,-0.1223,...,-0.0816,3.3918,1.6233,5.4834,0.9293,10.0405,2.7641,9.2344,6.0643,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316,-201.3727,-3.3076,-6.1641,31.8192,6.1636,-46.7698,-10.1254,34.2092,5.4326,2.6181,...,16.3118,-14.8115,3.0369,6.4507,-2.8558,-23.0134,-6.0959,-16.8932,31.8278,0.0
319,-3.4889,-7.5468,5.6223,-42.8946,-0.5838,-7.7837,-24.9255,-8.1583,-34.5417,-12.6059,...,45.4875,49.4695,27.1290,62.0512,47.4708,13.4493,24.4128,29.2915,37.7464,0.0
320,-7.9270,-6.4717,41.8189,2.2906,1.1288,13.5942,-28.7225,-5.6220,1357.4370,-0.0739,...,-15.9046,-15.5793,-21.8264,-35.8572,-5.2574,-8.3024,-20.7946,-4.7600,-6.7358,2.0
321,-4708.2950,10.6987,20.6212,-31.9737,0.1758,17.5586,28.2019,14.6711,64.9319,3.1740,...,-0.5512,-46.3319,-28.5157,7.1775,-11.4533,-31.7425,-45.8124,-20.0943,-45.0598,0.0


In [78]:
mapping_dict = {idx: i for i, idx in enumerate(dataDF.index[:244])}
dataDF.index = dataDF.index.map(mapping_dict)
dataDF

,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,17.8702,21.4625,7.5565,-27.3715,2.4686,4.5210,-4.7260,-12.1676,2.2848,-16.4927,...,16.4425,27.0572,-11.4758,4.2899,27.0235,-7.7163,-24.5729,3.5688,-1.5061,1.0
1,-34.6827,-13.2270,28.4124,37.2452,-28.3499,-30.9165,18.4352,-21.4872,18.3771,27.5069,...,-30.0969,13.4330,27.0925,-14.5877,46.8918,3.2198,19.4962,38.6907,-53.4658,0.0
2,-76.7901,10.4661,60.9218,39.1930,33.3773,65.0877,49.0782,5.1342,43.5867,0.9796,...,-38.8314,-50.8898,-42.2924,-22.9204,-48.7766,-44.7011,-38.4557,-40.1042,-36.8672,1.0
3,4.3084,92.5663,84.6904,46.8202,48.0335,51.5039,53.3825,32.5205,93.0373,2.1225,...,14.2065,10.0970,24.9994,33.3981,-4.9856,43.4911,34.7056,-2.6518,-3.7837,1.0
4,1.3368,-22.5723,15.9681,-59.4414,-10.9613,-30.8094,-60.6815,-26.9298,220.6445,-0.1223,...,-0.0816,3.3918,1.6233,5.4834,0.9293,10.0405,2.7641,9.2344,6.0643,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,-201.3727,-3.3076,-6.1641,31.8192,6.1636,-46.7698,-10.1254,34.2092,5.4326,2.6181,...,16.3118,-14.8115,3.0369,6.4507,-2.8558,-23.0134,-6.0959,-16.8932,31.8278,0.0
239,-3.4889,-7.5468,5.6223,-42.8946,-0.5838,-7.7837,-24.9255,-8.1583,-34.5417,-12.6059,...,45.4875,49.4695,27.1290,62.0512,47.4708,13.4493,24.4128,29.2915,37.7464,0.0
240,-7.9270,-6.4717,41.8189,2.2906,1.1288,13.5942,-28.7225,-5.6220,1357.4370,-0.0739,...,-15.9046,-15.5793,-21.8264,-35.8572,-5.2574,-8.3024,-20.7946,-4.7600,-6.7358,2.0
241,-4708.2950,10.6987,20.6212,-31.9737,0.1758,17.5586,28.2019,14.6711,64.9319,3.1740,...,-0.5512,-46.3319,-28.5157,7.1775,-11.4533,-31.7425,-45.8124,-20.0943,-45.0598,0.0


In [79]:
ct = 500

x_val = []
y_val = []

print(len(dataDF))

for i in range(len(dataDF)):
    print(i)
    x_val.append([])
    x = 0
    y = 20
    one_row_data = dataDF.iloc[i]
    y_val.append(one_row_data[y - 1])
    for j in range(ct):
        x_val[i].append([one_row_data[x:y - 1]])
        x += 20
        y += 20

x_val = np.array(x_val).reshape(len(x_val), ct, 19)
y_val = np.array(pd.get_dummies(y_val))

243
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242


In [80]:
print("Validation set shapes:")
print(x_val.shape)
print(y_val.shape)

Validation set shapes:
(243, 500, 19)
(243, 3)


In [81]:
# # 데이터 증강 함수 정의
# def augment_data(x, y):
#     augmented_x = []
#     augmented_y = []

#     for i in range(len(x)):
#         augmented_x.append(x[i])
#         augmented_y.append(y[i])


#         # 데이터를 뒤집기
#         flipped_x = np.flip(x[i], axis=0)
#         augmented_x.append(flipped_x)
#         augmented_y.append(y[i])

#         # 노이즈 추가
#         noisy_x = x[i] + np.random.normal(0, 0.1, x[i].shape)
#         augmented_x.append(noisy_x)
#         augmented_y.append(y[i])

#         # 주파수 변환 (예시: 500Hz에서 250Hz로 다운샘플링)
#         downsampled_x = resample(x[i], int(x[i].shape[0] / 2))

#         downsampled_x = np.pad(downsampled_x, ((0, x[i].shape[0] - downsampled_x.shape[0]), (0, 0)), mode='constant')

#         augmented_x.append(downsampled_x)
#         augmented_y.append(y[i])

#     return np.array(augmented_x), np.array(augmented_y)

# augmented_x_val, augmented_y_val = augment_data(x_val, y_val)


# print("\n데이터 증강 후 shape:")
# print("x_val shape:", augmented_x_val.shape)
# print("y_val shape:", augmented_y_val.shape)

In [82]:
path='/content/drive/MyDrive/Capstone_Design/EEG_DATA_Shuffle_Test_500.csv'
dataDF = pd.read_csv(path)
dataDF = dataDF.drop('Unnamed: 0', axis=1)
dataDF = dataDF[dataDF.iloc[:, -1] != 3]
dataDF

,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,10.4428,-6.2617,-17.0080,-26.7763,-2.7437,-8.7904,-1.2462,4.5911,0.7072,5.4368,...,25.0777,-0.0593,21.1675,4.0514,9.1570,17.9326,24.1978,32.4984,34.7287,1.0
1,27.5277,9.1181,-5.3722,15.0280,-11.4663,5.6988,-3.4859,49.7776,25.7247,-20.5147,...,23.8738,43.8601,13.6545,0.7813,31.9035,52.5582,37.7786,-17.5178,13.9098,1.0
2,3.5141,-46.9753,-42.6202,-10.6172,-6.7741,-12.3721,-12.9584,-55.8731,-11.5008,11.3118,...,-11.1626,-5.3945,-22.7586,16.5264,-6.6955,5.2774,5.3839,-28.1251,-27.5721,2.0
4,14.2301,31.4379,32.5355,9.3517,20.1011,28.8178,25.1274,7.3163,1.7966,6.4908,...,-6.5038,-7.1040,-14.3001,10.0207,-5.7362,-6.8697,-6.8623,-4.3336,-4.5961,2.0
7,-0.8922,-11.1875,-22.7758,-26.9243,-6.6699,-19.6985,-9.1861,-29.2356,-31.9603,-14.2397,...,-6.0263,-0.4245,-17.3971,-24.2367,-56.6827,-26.7129,4.9124,-28.6875,-23.7476,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,-18.4702,5.9324,1.0484,-33.7158,-0.2359,-19.5916,1.0997,6.5314,-24.6718,-27.4875,...,0.1658,13.0469,-1.3767,8.8103,8.8856,25.7677,-10.2174,4.2981,35.6747,1.0
211,2.9216,-20.6072,-8.2800,-13.4839,-2.1357,0.4423,-0.6483,-3.5984,7.6571,-18.1392,...,26.0548,22.5790,17.8109,15.7244,18.7196,34.5459,21.9128,18.4194,11.4716,1.0
212,-19.6713,-28.8127,-14.9236,2.2462,-2.1703,-3.0315,19.7007,0.3452,-12.7320,2.0537,...,-12.2667,13.6936,-0.2940,-1.6649,-5.2100,-2.0658,-10.6158,-12.2835,-13.5389,0.0
213,25.8002,7.1440,21.1307,17.3725,18.2719,13.6507,11.7411,22.2569,23.6596,21.1655,...,32.2828,-4.7560,-5.1947,-0.0218,7.3851,13.6946,-5.2101,7.4650,15.2721,1.0


In [83]:
mapping_dict = {idx: i for i, idx in enumerate(dataDF.index[:163])}
dataDF.index = dataDF.index.map(mapping_dict)
dataDF

,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,10.4428,-6.2617,-17.0080,-26.7763,-2.7437,-8.7904,-1.2462,4.5911,0.7072,5.4368,...,25.0777,-0.0593,21.1675,4.0514,9.1570,17.9326,24.1978,32.4984,34.7287,1.0
1,27.5277,9.1181,-5.3722,15.0280,-11.4663,5.6988,-3.4859,49.7776,25.7247,-20.5147,...,23.8738,43.8601,13.6545,0.7813,31.9035,52.5582,37.7786,-17.5178,13.9098,1.0
2,3.5141,-46.9753,-42.6202,-10.6172,-6.7741,-12.3721,-12.9584,-55.8731,-11.5008,11.3118,...,-11.1626,-5.3945,-22.7586,16.5264,-6.6955,5.2774,5.3839,-28.1251,-27.5721,2.0
3,14.2301,31.4379,32.5355,9.3517,20.1011,28.8178,25.1274,7.3163,1.7966,6.4908,...,-6.5038,-7.1040,-14.3001,10.0207,-5.7362,-6.8697,-6.8623,-4.3336,-4.5961,2.0
4,-0.8922,-11.1875,-22.7758,-26.9243,-6.6699,-19.6985,-9.1861,-29.2356,-31.9603,-14.2397,...,-6.0263,-0.4245,-17.3971,-24.2367,-56.6827,-26.7129,4.9124,-28.6875,-23.7476,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,-18.4702,5.9324,1.0484,-33.7158,-0.2359,-19.5916,1.0997,6.5314,-24.6718,-27.4875,...,0.1658,13.0469,-1.3767,8.8103,8.8856,25.7677,-10.2174,4.2981,35.6747,1.0
158,2.9216,-20.6072,-8.2800,-13.4839,-2.1357,0.4423,-0.6483,-3.5984,7.6571,-18.1392,...,26.0548,22.5790,17.8109,15.7244,18.7196,34.5459,21.9128,18.4194,11.4716,1.0
159,-19.6713,-28.8127,-14.9236,2.2462,-2.1703,-3.0315,19.7007,0.3452,-12.7320,2.0537,...,-12.2667,13.6936,-0.2940,-1.6649,-5.2100,-2.0658,-10.6158,-12.2835,-13.5389,0.0
160,25.8002,7.1440,21.1307,17.3725,18.2719,13.6507,11.7411,22.2569,23.6596,21.1655,...,32.2828,-4.7560,-5.1947,-0.0218,7.3851,13.6946,-5.2101,7.4650,15.2721,1.0


In [84]:
ct = 500

x_test = []
y_test = []

print(len(dataDF))

for i in range(len(dataDF)):
    print(i)
    x_test.append([])
    x = 0
    y = 20
    one_row_data = dataDF.iloc[i]
    y_test.append(one_row_data[y - 1])
    for j in range(ct):
        x_test[i].append([one_row_data[x:y - 1]])
        x += 20
        y += 20

x_test = np.array(x_test).reshape(len(x_test), ct, 19)
y_test = np.array(pd.get_dummies(y_test))

162
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161


In [85]:
print("Test set shapes:")
print(x_test.shape)
print(y_test.shape)

Test set shapes:
(162, 500, 19)
(162, 3)


In [86]:
# # 데이터 증강 함수 정의
# def augment_data(x, y):
#     augmented_x = []
#     augmented_y = []

#     for i in range(len(x)):
#         augmented_x.append(x[i])
#         augmented_y.append(y[i])

#         # 데이터를 뒤집기
#         flipped_x = np.flip(x[i], axis=0)
#         augmented_x.append(flipped_x)
#         augmented_y.append(y[i])

#         # 노이즈 추가
#         noisy_x = x[i] + np.random.normal(0, 0.1, x[i].shape)
#         augmented_x.append(noisy_x)
#         augmented_y.append(y[i])

#         # 주파수 변환 (예시: 500Hz에서 250Hz로 다운샘플링)
#         downsampled_x = resample(x[i], int(x[i].shape[0] / 2))

#         downsampled_x = np.pad(downsampled_x, ((0, x[i].shape[0] - downsampled_x.shape[0]), (0, 0)), mode='constant')

#         augmented_x.append(downsampled_x)
#         augmented_y.append(y[i])

#     return np.array(augmented_x), np.array(augmented_y)

# augmented_x_test, augmented_y_test = augment_data(x_test, y_test)

# print("\n데이터 증강 후 shape:")
# print("x_test shape:", augmented_x_test.shape)
# print("y_test shape:", augmented_y_test.shape)

In [87]:
# def transformer_block(input, dropout_rate=0.6):
#     x = keras.layers.MultiHeadAttention(num_heads=16, key_dim=1, dropout=dropout_rate)(input, input, return_attention_scores=False)
#     x = keras.layers.Dropout(dropout_rate)(x)
#     x = keras.layers.LayerNormalization(epsilon=1e-6)(x + input)
#     x2 = keras.layers.Dense(19, activation='relu')(x)
#     x2 = keras.layers.Dropout(dropout_rate)(x2)
#     x = keras.layers.LayerNormalization(epsilon=1e-6)(x2 + x)
#     return x

In [88]:
# def build_model(num_block, dropout_rate=0.6):
#     inputs = keras.Input((ct, 19))
#     x = inputs
#     for _ in range(num_block):
#         x = transformer_block(x, dropout_rate=dropout_rate)
#     x = keras.layers.GlobalAveragePooling1D()(x)
#     output = keras.layers.Dense(4, activation='softmax')(x)
#     return keras.Model(inputs, output)

In [96]:
# num_block = 4
# dropout_rate = 0.6
# myMdl = build_model(num_block, dropout_rate)
# myMdl.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_6 (InputLayer)        [(None, 500, 19)]            0         []                            
                                                                                                  
 multi_head_attention_17 (M  (None, 500, 19)              1283      ['input_6[0][0]',             
 ultiHeadAttention)                                                  'input_6[0][0]']             
                                                                                                  
 dropout_34 (Dropout)        (None, 500, 19)              0         ['multi_head_attention_17[0][0
                                                                    ]']                           
                                                                                            

In [103]:
def transformer_block(input):
  x = keras.layers.MultiHeadAttention(num_heads=16, key_dim=1, dropout=0.6)(input, input, return_attention_scores=False)
  x = x + input
  x = keras.layers.LayerNormalization()(x)
  x2 = keras.layers.Dense(19,activation='relu')(x)
  x2 = x + x2
  x3 = keras.layers.LayerNormalization()(x2)
  return x3

In [115]:
def build_model(num_block):
  inputs = keras.Input((ct,19))
  x = inputs
  for _ in range(num_block):
    x = transformer_block(x)
  x = keras.layers.GlobalAveragePooling1D()(x)
  output = keras.layers.Dense(3,activation='softmax')(x)
  return keras.Model(inputs,output)

In [116]:
num_block = 4
myMdl = build_model(num_block)
myMdl.summary()

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_9 (InputLayer)        [(None, 500, 19)]            0         []                            
                                                                                                  
 multi_head_attention_29 (M  (None, 500, 19)              1283      ['input_9[0][0]',             
 ultiHeadAttention)                                                  'input_9[0][0]']             
                                                                                                  
 tf.__operators__.add_58 (T  (None, 500, 19)              0         ['multi_head_attention_29[0][0
 FOpLambda)                                                         ]',                           
                                                                     'input_9[0][0]']       

In [118]:
root = '/content/drive/MyDrive/Capstone_Design/Transformer_Save_Model/'
folder_name = '0417_1modelsave/'
#os.mkdir(root+folder_name)
csv_logger = CSVLogger(root+folder_name+'/training.csv', separator=",", append=True)
mcp_save_best = tf.keras.callbacks.ModelCheckpoint(
    root+folder_name+'/{epoch:02d}-{val_loss:.5f}_best', save_best_only=True,
    monitor='val_loss', verbose=1, mode='min')
mcp_save = tf.keras.callbacks.ModelCheckpoint(
    root+folder_name+'/{epoch:02d}-{val_loss:.5f}', save_best_only=False,
    monitor='val_loss', verbose=1, mode='min')

In [119]:
adam = Adam(learning_rate=0.00001)

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, restore_best_weights=True)
mc_loss = ModelCheckpoint('/content/drive/MyDrive/Capstone_Design/Transformer_Save_Model/best_transformer_loss_v1.h5', monitor='val_loss', verbose=1, save_best_only=True)

mc = ModelCheckpoint('/content/drive/MyDrive/Capstone_Design/Transformer_Save_Model/best_transformer_acc_v1.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

lr_schedule = LearningRateScheduler(lambda epoch: 0.001 * np.exp(-epoch / 10.))

myMdl.compile(optimizer=adam, loss=['categorical_crossentropy'], metrics=['accuracy'])

trans_h = myMdl.fit(x_train, y_train,
                   batch_size=32,
                   validation_data=(x_val, y_val),
                   epochs=200,
                   callbacks=[es, mc, mc_loss,mcp_save_best,mcp_save, lr_schedule, csv_logger])

Epoch 1/200
51/51 [==============================] - ETA: 0s - loss: 0.9288 - accuracy: 0.5685
Epoch 1: val_accuracy improved from -inf to 0.67078, saving model to /content/drive/MyDrive/Capstone_Design/Transformer_Save_Model/best_transformer_acc_v1.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 1: val_loss improved from inf to 1.13336, saving model to /content/drive/MyDrive/Capstone_Design/Transformer_Save_Model/best_transformer_loss_v1.h5

Epoch 1: val_loss improved from inf to 1.13336, saving model to /content/drive/MyDrive/Capstone_Design/Transformer_Save_Model/0417_1modelsave/01-1.13336_best

Epoch 1: saving model to /content/drive/MyDrive/Capstone_Design/Transformer_Save_Model/0417_1modelsave/01-1.13336
51/51 [==============================] - 38s 556ms/step - loss: 0.9288 - accuracy: 0.5685 - val_loss: 1.1334 - val_accuracy: 0.6708 - lr: 0.0010
Epoch 2/200
51/51 [==============================] - ETA: 0s - loss: 0.5848 - accuracy: 0.8093
Epoch 2: val_accuracy did not improve from 0.67078

Epoch 2: val_loss did not improve from 1.13336

Epoch 2: val_loss did not improve from 1.13336

Epoch 2: saving model to /content/drive/MyDrive/Capstone_Design/Transformer_Save_Model/0417_1modelsave/02-1.21780
51/51 [==============================] - 20s 400ms/step - loss: 0.584

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 5: val_loss improved from 1.13336 to 1.07505, saving model to /content/drive/MyDrive/Capstone_Design/Transformer_Save_Model/0417_1modelsave/05-1.07505_best

Epoch 5: saving model to /content/drive/MyDrive/Capstone_Design/Transformer_Save_Model/0417_1modelsave/05-1.07505
51/51 [==============================] - 25s 501ms/step - loss: 0.2004 - accuracy: 0.9290 - val_loss: 1.0751 - val_accuracy: 0.6173 - lr: 6.7032e-04
Epoch 6/200
51/51 [==============================] - ETA: 0s - loss: 0.1503 - accuracy: 0.9395
Epoch 6: val_accuracy did not improve from 0.67078

Epoch 6: val_loss did not improve from 1.07505

Epoch 6: val_loss did not improve from 1.07505

Epoch 6: saving model to /content/drive/MyDrive/Capstone_Design/Transformer_Save_Model/0417_1modelsave/06-1.11565
51/51 [==============================] - 116s 2s/step - loss: 0.1503 - accuracy: 0.9395 - val_loss: 1.1156 - val_accuracy: 0.6337 - lr: 6.0653e-04
Epoch 7/200
28/51 [===============>..............] - ETA: 6s - loss: 

KeyboardInterrupt: 

In [ ]:
# adam = Adam(learning_rate=0.001)

# es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, restore_best_weights=True)
# mc_loss = ModelCheckpoint('/content/drive/MyDrive/Capstone_Design/Transformer_Save_Model/best_transformer_loss_v1.h5', monitor='val_loss', verbose=1, save_best_only=True)

# mc = ModelCheckpoint('/content/drive/MyDrive/Capstone_Design/Transformer_Save_Model/best_transformer_acc_v1.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

# lr_schedule = LearningRateScheduler(lambda epoch: 0.001 * np.exp(-epoch / 10.))

# myMdl.compile(optimizer=adam, loss=['categorical_crossentropy'], metrics=['accuracy'])
# # augmented_x_train, augmented_y_train
# trans_h = myMdl.fit(augmented_x_train, augmented_y_train,
#                    batch_size=32,
#                    validation_data=(augmented_x_val, augmented_y_val),
#                    epochs=200,
#                    callbacks=[es, mc, mc_loss,mcp_save_best,mcp_save, lr_schedule, csv_logger])

In [ ]:
# transformer_best = load_model('/content/drive/MyDrive/Capstone_Design/Save_Model/best_transformer_acc_v1.h5')
# transformer_accuracy = transformer_best.evaluate(x_test, y_test, verbose=1)[1]
# print("Test Acc. Transformer: {:.3f}%".format(transformer_accuracy * 100))

# transformer_best_loss = load_model('/content/drive/MyDrive/Capstone_Design/Save_Model/best_transformer_loss_v1.h5')
# transformer_loss = transformer_best_loss.evaluate(x_test, y_test, verbose=1)[0]
# print("Test val_Loss. transformer:{:.3f}".format(transformer_loss))

# y_pred = np.array(list(map(lambda x: np.argmax(x), transformer_best.predict(x_test))))
# y_test = y_test.argmax(axis=1)
# cm = confusion_matrix(y_test, y_pred)
# print(classification_report(y_test, y_pred, digits=4))

In [ ]:
# y_pred_one_hot = transformer_best.predict(x_test)
# y_pred = np.argmax(y_pred_one_hot, axis=1)

# lb = LabelBinarizer()
# lb.fit(y_test)
# y_test_one_hot = lb.transform(y_test)


# fpr = dict()
# tpr = dict()
# roc_auc = dict()
# class_labels = ['Eyes Open', 'Eyes Closed', 'Waving hand', 'Mental arithmetic']

# for i in range(len(class_labels)):
#     fpr[i], tpr[i], _ = roc_curve(y_test_one_hot[:, i], y_pred_one_hot[:, i])
#     roc_auc[i] = auc(fpr[i], tpr[i])

# all_fpr = np.unique(np.concatenate([fpr[i] for i in range(len(class_labels))]))
# mean_tpr = np.zeros_like(all_fpr)
# for i in range(len(class_labels)):
#     mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
# mean_tpr /= len(class_labels)

# fpr_macro = all_fpr
# tpr_macro = mean_tpr
# roc_auc_macro = auc(fpr_macro, tpr_macro)

# plt.figure()
# plt.plot(fpr_macro, tpr_macro,
#          label='macro-average (area = {0:0.4f})'
#                ''.format(roc_auc_macro),
#          color='navy', linestyle=':', linewidth=4)

# for i in range(len(class_labels)):
#     plt.plot(fpr[i], tpr[i], lw=2,
#              label='{0} (area = {1:0.4f})'
#              ''.format(class_labels[i], roc_auc[i]))

# plt.plot([0, 1], [0, 1], 'k--', lw=2)
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Transformer - ROC CURVE')
# plt.legend(loc="lower right")
# plt.show()

In [ ]:
# myMdl = tf.keras.models.load_model('/content/drive/MyDrive/캡스톤디자인I/Real_Save_Model/20240416_7modelsave/12-0.33696_best')

In [ ]:
# history = pd.read_csv('/content/drive/MyDrive/캡스톤디자인I/Real_Save_Model/20240416_7modelsave/training.csv')

In [ ]:
# acc = trans_h.history['accuracy']
# val_acc = trans_h.history['val_accuracy']
# loss = trans_h.history['loss']
# val_loss = trans_h.history['val_loss']
# plt.figure(figsize=(12, 12))
# plt.subplot(2, 1, 1)
# plt.plot(acc, label='Training Accuracy', color='r')
# plt.plot(val_acc, label='Validation Accuracy', color='b', linestyle='--')
# plt.xticks(fontsize=14)
# plt.yticks(fontsize=14)
# plt.legend(loc='lower right', fontsize=13)
# plt.ylabel('Accuracy', fontsize=16, weight='bold')
# plt.title('Transformer - Training & Validation Acc.', fontsize=16, weight='bold')
# plt.xlabel('Epoch', fontsize=15, weight='bold')
# plt.subplot(2, 1, 2)
# plt.plot(loss, label='Training Loss', color='r')
# plt.plot(val_loss, label='Validation Loss', color='b', linestyle='--')
# plt.xticks(fontsize=14)
# plt.yticks(fontsize=14)
# plt.legend(loc='upper right', fontsize=13)
# plt.ylabel('Cross Entropy', fontsize=16, weight='bold')
# plt.title('Transformer - Training & Validation Loss', fontsize=15, weight='bold')
# plt.xlabel('Epoch', fontsize=15, weight='bold')
# plt.show()